# **Módulo 4 (Estratégias de treinamento e transferência de aprendizado)** 

**Prática 1:** Extração de características com ResNet50 pré-treinada, redução de dimensionalidade, fusão de atributos e classificação com SVM

Antes de iniciar a execução do código faça a seguinte alteração:
Menu: **Ambiente de execução -> Alterar o tipo de ambiente de execução -> GPU**

In [1]:
import numpy as np
from PIL import Image

from keras.datasets import cifar10
from keras.models import Model
from keras.applications import resnet50

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

A função **lowSampleDataset(X,Y)** apenas reduz a quantidade de exemplos para que a execução seja mais rápida em nosso exemplo!

In [2]:
def lowSampleDataset(X, Y):
    perm = np.random.permutation(X.shape[0])
    X = X[perm[0 : (int)(X.shape[0] * (5/100))]]
    Y = Y[perm[0 : (int)(Y.shape[0] * (5/100))]]
    return X, Y

**Pré-processamento:**

1.   Carregamos o dataset CIFAR10
2.   Reduzimos a quantidade de exemplos
1.   Alteramos a resolução dos exemplos de acordo com o input da ResNet50

In [3]:
print("Loading CIFAR10 images ...")
(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()

print('\tOriginal training set shape: ', Xtrain.shape)
print('\tOriginal testing set shape: ', Xtest.shape)

Xtrain, Ytrain = lowSampleDataset(Xtrain, Ytrain)
Xtest, Ytest = lowSampleDataset(Xtest, Ytest)

X = []
for i in range(0, Xtrain.shape[0]):
    X.append(np.array(Image.fromarray(Xtrain[i]).resize(size=(224,224))))
Xtrain = np.array(X)

X = []
for i in range(0, Xtest.shape[0]):
    X.append(np.array(Image.fromarray(Xtest[i]).resize(size=(224,224))))
Xtest = np.array(X)

print('\tTraining set shape: ', Xtrain.shape)
print('\tTesting set shape: ', Xtest.shape)

Loading CIFAR10 images ...
	Original training set shape:  (50000, 32, 32, 3)
	Original testing set shape:  (10000, 32, 32, 3)
	Training set shape:  (2500, 224, 224, 3)
	Testing set shape:  (500, 224, 224, 3)


**Extração de características:**

Carregamos a ResNet50 com os pesos da ImageNet e extraímos características utilizando a camada de pré-predição (-2)

In [4]:
print("Loading the ResNet50-ImageNet model ...")
model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
#model.summary()

model1 = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)

prediction = np.array(model1.predict(Xtrain))
Ftrain1 = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

prediction = np.array(model1.predict(Xtest))
Ftest1 = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', Ftrain1.shape)
print('\tFeatures testing shape: ', Ftest1.shape)

Loading the ResNet50-ImageNet model ...
	Features training shape:  (2500, 2048)
	Features testing shape:  (500, 2048)


**Nova extração de características:**

A partir da ResNet50 com os pesos da ImageNet e extraímos características utilizando a camada anterior à de pré-predição (-3)

In [5]:
model2 = Model(inputs=model.input, outputs=model.get_layer(index=-3).output)

prediction = np.array(model2.predict(Xtrain))
Ftrain2 = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]*prediction.shape[2]*prediction.shape[3]))

prediction = np.array(model2.predict(Xtest))
Ftest2 = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]*prediction.shape[2]*prediction.shape[3]))

print('\tFeatures training shape: ', Ftrain2.shape)
print('\tFeatures testing shape: ', Ftest2.shape)

	Features training shape:  (2500, 100352)
	Features testing shape:  (500, 100352)


**Redução de dimensionalidade:**

As características extraídas da camada (-2) possuem dimensão de 2048. Em alguns casos, a dimensionalidade pode ser muito alta, acarretando em muito processamento, como a camada (-3) com 100352 atributos. Uma solução para agilizar o processo é a redução de dimensionalidade.

In [6]:
print("Dimensionality reduction with PCA ...")
pca = PCA(n_components=256)
Ftrain1 = pca.fit_transform(Ftrain1)
Ftest1 = pca.transform(Ftest1)

print('\tFeatures training 1 shape: ', Ftrain1.shape)
print('\tFeatures testing 1 shape: ', Ftest1.shape)

print("Dimensionality reduction with PCA ...")
Ftrain2 = pca.fit_transform(Ftrain2)
Ftest2 = pca.transform(Ftest2)

print('\tFeatures training 2 shape: ', Ftrain2.shape)
print('\tFeatures testing 2 shape: ', Ftest2.shape)

Dimensionality reduction with PCA ...
	Features training 1 shape:  (2500, 256)
	Features testing 1 shape:  (500, 256)
Dimensionality reduction with PCA ...
	Features training 2 shape:  (2500, 256)
	Features testing 2 shape:  (500, 256)


**Classificação:**

Vamos aplicar o SVM a cada um dos conjuntos de características separadamente, ou seja, primeiramente para a camada -2 e, logo após, para a camada -3.

In [7]:
print("Classification with Linear SVM ...")
svm = SVC(kernel='linear')
svm.fit(Ftrain1, np.ravel(Ytrain, order='C'))
result = svm.predict(Ftest1)

acc = accuracy_score(result, np.ravel(Ytest, order='C'))
print("\tAccuracy Linear 1 SVM: %0.4f" % acc)

svm.fit(Ftrain2, np.ravel(Ytrain, order='C'))
result = svm.predict(Ftest2)

acc = accuracy_score(result, np.ravel(Ytest, order='C'))
print("\tAccuracy Linear 2 SVM: %0.4f" % acc)

Classification with Linear SVM ...
	Accuracy Linear 1 SVM: 0.7660
	Accuracy Linear 2 SVM: 0.8060


**Fusão de atributos:**
Tambem podemos realizar a fusão de mapas de características, por meio da concatenação ou qualquer outro processo.

In [8]:
Ftrain3 = np.concatenate((Ftrain1, Ftrain2), axis=1)
Ftest3 = np.concatenate((Ftest2, Ftest2), axis=1)

Ftrain3 = pca.fit_transform(Ftrain3)
Ftest3 = pca.transform(Ftest3)

print('\tFeatures training 3 shape: ', Ftrain3.shape)
print('\tFeatures testing 3 shape: ', Ftest3.shape)

svm.fit(Ftrain3, np.ravel(Ytrain, order='C'))
result = svm.predict(Ftest3)

acc = accuracy_score(result, np.ravel(Ytest, order='C'))
print("\tAccuracy Linear 3 SVM: %0.4f" % acc)


	Features training 3 shape:  (2500, 256)
	Features testing 3 shape:  (500, 256)
	Accuracy Linear 3 SVM: 0.7940


**Dicas:**

1.   É altamente recomendável investigar camadas anteriores à de pré-predição, pois podem apresentar melhor performance que a camada de pré-predição. Comumente na literatura, essas camadas são ignoradas.
2.   É observável em experimentos que datasets com aparência visual de baixo nível (bordas e cores, por exemplo) podem ser beneficiar com a extração de características de camadas iniciais. Entretanto, essas camadas possuem alta dimensionalidade e, consequentemente, requerem alto poder computacional.
1.   A partir da extração de características diversos processos podem ser aplicados às características: redução de dimensionalidade, fusão, pós-processamento com alinhamento e, até, mesmo como entrada de MLPs.

